### Aprendizaje por clasificación

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
len(df)

### Preparación inicial de los datos

In [ ]:
df.head()

In [ ]:
df.head().T

In [ ]:
df.dtypes

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

cadena_columnas = list(df.dtypes[df.dtypes == 'object'].index)

for col in cadena_columnas:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [ ]:
df.churn = (df.churn == 'yes').astype(int)

In [ ]:
df.head().T

### Uso de librerías scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train_completo, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
df_train, df_val = train_test_split(df_train_completo, test_size=0.33, random_state=11)

In [ ]:
y_train = df_train.churn.values
y_val = df_val.churn.values

In [ ]:
del df_train['churn']
del df_val['churn']

### EDA

In [ ]:
df_train_completo.isnull().sum()

 Verificamos la distribución de valores en la variable objetivo.

In [ ]:
df_train_completo.churn.value_counts()

Calcular la tasa de abandono de clientes: el método `mean()`. 

In [ ]:
media_global = df_train_completo.churn.mean()
round(media_global, 3)

Las variables categóricas como las numéricas en el conjunto de datos son importantes, pero también son diferentes y necesitan un tratamiento diferente.

In [ ]:
categoricas = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numericas = ['tenure', 'monthlycharges', 'totalcharges']


Imprimamos cuántos valores únicos tiene cada variable.


In [ ]:
df_train_completo[categoricas].nunique()

### Importancia de características

#### Tasa de abandono

Comprobemos primero la variable `gender`.  Esta variable `gender` puede tomar dos valores `female` y  `male`. Hay dos grupos de clientes: los que tienen `gender == 'female'` y los que tienen `gender == 'male'`.

In [ ]:
media_female = df_train_completo[df_train_completo.gender == 'female'].churn.mean()
print('gender == female:', round(media_female, 3))

media_male = df_train_completo[df_train_completo.gender == 'male'].churn.mean()
print('gender == male:  ', round(media_male, 3))

In [ ]:
media_female / media_global

In [ ]:
media_male/media_global

Ahora echemos un vistazo a otra variable: `partner`. Toma valores de `yes`  y `no`, por lo que hay dos grupos de clientes: aquellos para los que `partner == 'yes'` y aquellos para los que `partner == 'no'`. 


In [ ]:
si_partner = df_train_completo[df_train_completo.partner == 'yes'].churn.mean()
print('partner == yes:', round(si_partner, 3))

no_partner = df_train_completo[df_train_completo.partner == 'no'].churn.mean()
print('partner == no :', round(no_partner, 3))

In [ ]:
si_partner /media_global

In [ ]:
no_partner /media_global

#### Tasa de riesgo

Calculemos los riesgos por `gender` en el conjunto de datos. 




In [ ]:
df_grupo = df_train_completo.groupby(by='gender').churn.agg(['mean'])
df_grupo['diff'] = df_grupo['mean'] - media_global
df_grupo['risk'] = df_grupo['mean'] / media_global
df_grupo

Ahora hagamos eso para todas las variables categóricas. Podemos iterar a través de ellos y aplicar el mismo código para cada uno: 

In [ ]:
from IPython.display import display

In [ ]:
media_global = df_train_completo.churn.mean()
media_global

In [ ]:
for col in categoricas:
    df_grupo = df_train_completo.groupby(by=col).churn.agg(['mean'])
    df_grupo['diff'] = df_grupo['mean'] - media_global
    df_grupo['risk'] = df_grupo['mean'] / media_global
    #display(df_grupo)

### Información mutua

Para las variables categóricas indica cuánta información aprendemos sobre una variable si aprendemos el valor de la otra variable.

In [ ]:
from sklearn.metrics import mutual_info_score

In [ ]:
def calcula_mi(series):
    return mutual_info_score(series, df_train_completo.churn)

df_mi = df_train_completo[categoricas].apply(calcula_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head())
display(df_mi.tail())


#### Coeficiente de correlación

Calculemos el coeficiente de correlación en Pandas.

In [ ]:
df_train_completo[numericas].corrwith(df_train_completo.churn).to_frame('correlation')

In [ ]:
#df_train_completo.groupby(by='churn')[numericas].mean()

### Ingeniería de características

#### Codificación one-hot para variables categóricas


In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
train_dict = df_train[categoricas + numericas].to_dict(orient='records')

In [ ]:
train_dict[0]

Ahora podemos usar `DictVectorizer`. Lo creamos y luego lo ajustamos a la lista de diccionarios que creamos previamente: 


In [ ]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

Después de ajustar el vectorizador, podemos usarlo para convertir los diccionarios en una matriz usando el método `transform`: 

In [ ]:
X_train = dv.transform(train_dict)

In [ ]:
X_train.shape

In [ ]:
## X_train[0]

Podemos aprender los nombres de todas estas columnas usando el método `get_feature_names`: 


In [ ]:
import warnings
warnings.filterwarnings("ignore")
# # get_feature_names_out()
dv.get_feature_names() 

### Aprendizaje automático para clasificación 

#### Regresión logística


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
modeloRL = LogisticRegression(solver='liblinear', random_state=1)
modeloRL.fit(X_train, y_train)

Veamos qué tan bien se comporta el modelo.Para hacer eso, necesitamos aplicar el esquema de codificación one-hot a todas las variables categóricas.

In [ ]:
val_dict = df_val[categoricas + numericas].to_dict(orient='records')
X_val = dv.transform(val_dict)

Ahora estamos listos para poner esta matriz en el modelo. Para obtener las probabilidades, usamos el método `predict_proba` del modelo: 


In [ ]:
modeloRL.predict_proba(X_val)

**Ejercicio**

* Escribe solo la segunda columna de la predicción.
* Selecciona un umbral de 0.5 para realizar  predicciones binarias y calcula la exactitud del modelo.

In [ ]:
## Tus respuestas

###  Interpretación del modelo 

Calculamos el sesgo del modelo.

In [ ]:
modeloRL.intercept_[0]

Para ver qué característica está asociada con cada peso, usemos el método `get_feature_names` de `DictVectorizer`. Podemos comprimir los nombres de las características junto con los coeficientes antes de mirarlos: 

In [ ]:
dict(zip(dv.get_feature_names(), modeloRL.coef_[0].round(3))) # https://book.pythontips.com/en/latest/zip.html

Para comprender cómo funciona el modelo, entrenemos un modelo más simple y más pequeño que use solo tres variables: `contract` , `tenure` y `totalcharges`.

In [ ]:
subconjunto = ['contract', 'tenure', 'totalcharges']
train_dict_sub = df_train[subconjunto].to_dict(orient='records')
dv_sub = DictVectorizer(sparse=False)
dv_sub.fit(train_dict_sub)

X_sub_train = dv_sub.transform(train_dict_sub)
dv_sub.get_feature_names()

In [ ]:
submodeloRL = LogisticRegression(solver='liblinear', random_state=1)
submodeloRL.fit(X_sub_train, y_train)

In [ ]:
submodeloRL.intercept_[0]

In [ ]:
dict(zip(dv_sub.get_feature_names(), submodeloRL.coef_[0].round(3)))

In [ ]:
sub_val_dict = df_val[subconjunto].to_dict(orient='records')
X_sub_val = dv_sub.transform(sub_val_dict)

In [ ]:
sub_y_pred = submodeloRL.predict_proba(X_sub_val)[:, 1]

### Usando el modelo

Ahora podemos aplicar el modelo a los clientes para calificarlos.

In [ ]:
cliente = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75,
}

In [ ]:
X_test = dv.transform([cliente])
modeloRL.predict_proba(X_test)[0,1]

**Ejercicio** Realiza el mismo procedimiento y explica tu respuesta con el siguiente perfil de cliente.

In [ ]:
cliente = {
    'gender': 'female',
    'seniorcitizen': 1,
    'partner': 'no',
    'dependents': 'no',
    'phoneservice': 'yes',
    'multiplelines': 'yes',
    'internetservice': 'fiber_optic',
    'onlinesecurity': 'no',
    'onlinebackup': 'no',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'yes',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 85.7,
    'totalcharges': 85.7
}

In [ ]:
# Tu respuesta

### Ejercicios

1. Puedes probar un par de cosas para aprender mejor el tema: En el cuaderno anterior, implementamos muchas cosas nosotros mismos, incluida la regresión lineal y la división de conjuntos de datos. En este cuaderno aprendimos a usar Scikit-learn para eso. Intenta rehacer el proyecto del cuaderno  anterior usando Scikit-learn. Para usar la regresión lineal, necesita `LinearRegression` del paquete `sklearn.linear_model`. Para usar la regresión regularizada, debe importar `Ridge` desde el mismo paquete `sklearn.linear_model`. 

In [ ]:
## Tu respuesta

2. Analizamos las métricas de importancia de las características para obtener información sobre el conjunto de datos, pero en realidad no usamos esta información para otros fines. Una forma de usar esta información podría ser eliminar las características que no son útiles del conjunto de datos para hacer que el modelo sea más simple, más rápido y potencialmente mejor. Intenta excluir las dos características menos útiles (`gender` y `phoneservices`) de la matriz de datos de entrenamiento y observa qué sucede con la exactitud de la validación. ¿Qué pasa si eliminamos la característica más útil (`contract`)?.


In [ ]:
## Tu respuesta

3. Los modelos de clasificación se utilizan a menudo con fines de marketing y uno de los problemas que resuelve es la puntuación de clientes potenciales (`lead scoring`). Un cliente potencial (`lead`) es un cliente potencial que puede convertirse (convertirse en un cliente real) o no. En este caso, la conversión es el objetivo que queremos predecir. Puedes tomar un conjunto de datos de https://www.kaggle.com/ashydv/leads-dataset y crear un modelo para eso. Puedes notar que el problema de puntuación de clientes potenciales es similar a la predicción de abandono, pero en un caso, queremos que un nuevo cliente firme un contrato con nosotros y en otro caso, queremos que un cliente no cancele el contrato. 

In [ ]:
## Tu respuesta